# OpenAI API를 활용한 데이터 정제: 텍스트에서 구조화된 데이터까지

## 1. 기본 API 호출

### 1.1 라이브러리 및 객체 초기화

In [ ]:
OPENAI_API_KEY = "mykey"

In [3]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.11.0-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/948.6 kB ? eta -:--:--
   ---------------------------------------- 10.2/948.6 kB ? eta -:--:--
   - ------------------------------------- 41.0/948.6 kB 388.9 kB/s eta 0:00:03
   -------- ------------------------------- 204.8/948.6 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 948.6/948.6 kB 6.0 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.11.0-cp311-cp311-win_amd64.whl (204 kB)
Using cached pydant


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

### 1.2 OpenAI API 호출


In [ ]:
# 기본 API 호출 (주석 버전)
# OpenAI의 Responses API를 호출하여 응답을 생성합니다.
response = client.responses.create(
    # --- 사용할 AI 모델 지정 ---
    model="gpt-5-nano",
    # --- 모델에 전달할 입력 데이터 ---
    # 'input'은 시스템(system)과 사용자(user)의 역할을 담는 리스트 형태입니다.
    # 이를 통해 모델에게 대화의 맥락과 역할을 부여할 수 있습니다.
    input=[
        # 'system' 역할은 모델에게 기본적인 지침이나 역할을 부여합니다.
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    # 모델이 수행할 작업에 대한 전반적인 지시사항을 전달합니다.
                    # 예: "너는 비정형 데이터를 CSV 형식으로 변환하는 역할을 맡았어."
                    "text": "You will be provided with unstructured data, and your task is to parse it into CSV format.",
                }
            ],
        },
        # 'user' 역할은 모델이 처리해야 할 실제 데이터나 질문을 전달합니다.
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    # 모델이 CSV로 변환해야 할 원본 텍스트 데이터입니다.
                    "text": "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy. There are also loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them.",
                }
            ],
        },
    ],
)


In [8]:
# 기본 API 호출 (주석 제거 버전)
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    "text": "You will be provided with unstructured data, and your task is to parse it into CSV format.",
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy. There are also loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them.",
                }
            ],
        },
    ],
)


In [9]:
# 응답 출력
print(response.output_text)

Fruit,Color,TasteDescription
neoskizzles,purple,taste like candy
loheckles,grayish blue,tart; like a lemon
pounits,bright green,savory rather than sweet
loopnovas,neon pink,taste like cotton candy
glowls,pale orange tinge,very sour and bitter; acidic and caustic


## 2. 정형 데이터 처리: CSV 파싱

In [10]:
# 시스템 프롬프트 커스터마이징
system_prompt = """
당신은 구조화 되지 않은 데이터를 CSV 형식으로 변환하는 AI입니다
사용자가 과일에 대한 데이터를 제공하면, 과일 이름만 CSV 형식으로 추출하세요.
"""

In [11]:
# 유저 프롬프트 커스터마이징
user_prompt = """
한국 사람들이 좋아하는 과일은 사과, 배, 귤, 포도, 수박, 참외, 딸기, 감, 복숭아 같은 것들이 있어요. 
계절마다 선호하는 과일이 좀 다른데, 여름엔 수박이나 참외를 많이 먹고, 겨울엔 귤이 인기 많죠. 
딸기는 봄철에 특히 많이 찾고, 가을엔 감이나 배를 많이 먹어요. 
포도도 여름에서 가을 사이에 인기가 많고, 복숭아도 달콤하고 부드러워서 좋아하는 사람이 많아요. - 출처 : 00뉴스
"""

In [13]:
# 기본 API 호출 수정
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    "text": system_prompt,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": user_prompt,
                }
            ],
        },
    ],
)

In [14]:
# 응답 출력
print(response.output_text)

사과,배,귤,포도,수박,참외,딸기,감,복숭아


## 3. 계층형 데이터 처리: XML 파싱

In [15]:
# 시스템 프롬프트 커스터마이징
system_prompt = """
당신은 구조화 되지 않은 데이터를 CSV 형식으로 변환하는 AI입니다
사용자가 과일에 대한 데이터를 제공하면, 과일 이름만 XML 형식으로 추출하세요.
"""

In [16]:
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    "text": system_prompt,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": user_prompt,
                }
            ],
        },
    ],
)

In [17]:
# 응답 출력
print(response.output_text)

<fruits>
  <fruit>사과</fruit>
  <fruit>배</fruit>
  <fruit>귤</fruit>
  <fruit>포도</fruit>
  <fruit>수박</fruit>
  <fruit>참외</fruit>
  <fruit>딸기</fruit>
  <fruit>감</fruit>
  <fruit>복숭아</fruit>
</fruits>


## 4. 신뢰할 수 있는 JSON 생성하기

### 4.1 기본: 프롬프트로 JSON 형식 요청하기


In [18]:
# 시스템 프롬프트 수정
system_prompt = """
당신은 구조화 되지 않은 데이터를 JSON 형식으로 변환하는 AI입니다.
사용자가 과일에 대한 설명을 제공하면, 적잘한 키와 함께 과일 이름만 JSON 형식으로 추출하세요.
답변은 반드시 '{'로 시작하여 '}'로 끝나야 됩니다.
"""

In [19]:
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    "text": system_prompt,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": user_prompt,
                }
            ],
        },
    ],
)

In [20]:
# 응답 출력
print(response.output_text)

{
  "사과": {"season": null},
  "배": {"season": ["가을"]},
  "귤": {"season": ["겨울"]},
  "포도": {"season": ["여름", "가을"]},
  "수박": {"season": ["여름"]},
  "참외": {"season": ["여름"]},
  "딸기": {"season": ["봄"]},
  "감": {"season": ["가을"]},
  "복숭아": {"season": null}
}


In [ ]:
# json 받으면 그냥 못쓰기 때문에 파싱해서 쓰는것. 알고있어야함 
# 역으로 하는건 dumps.
# JSON 형식의 문자열을 파이썬 객체로 변환
# json => dict
import json

parsed_data = json.loads(response.output_text)
print(parsed_data)
print(type(parsed_data))

{'사과': {'season': None}, '배': {'season': ['가을']}, '귤': {'season': ['겨울']}, '포도': {'season': ['여름', '가을']}, '수박': {'season': ['여름']}, '참외': {'season': ['여름']}, '딸기': {'season': ['봄']}, '감': {'season': ['가을']}, '복숭아': {'season': None}}
<class 'dict'>


In [24]:
# 파이썬 객체를 JSON 형식의 문자열로 변환
# dict => json

# ensure_ascii=False는 한글과 같은 비(非)ASCII 문자가 \uc0ac\uacfc와 같은 유니코드 이스케이프 시퀀스로 변환되는 것을 막고, 
# 원본 문자 그대로 출력되도록 하는 역할
json_data = json.dumps(parsed_data, indent=4, ensure_ascii=False)

print(json_data)
print(type(json_data))

{
    "사과": {
        "season": null
    },
    "배": {
        "season": [
            "가을"
        ]
    },
    "귤": {
        "season": [
            "겨울"
        ]
    },
    "포도": {
        "season": [
            "여름",
            "가을"
        ]
    },
    "수박": {
        "season": [
            "여름"
        ]
    },
    "참외": {
        "season": [
            "여름"
        ]
    },
    "딸기": {
        "season": [
            "봄"
        ]
    },
    "감": {
        "season": [
            "가을"
        ]
    },
    "복숭아": {
        "season": null
    }
}
<class 'str'>


### 4.2 심화: 중첩된 JSON 구조 설계하기

In [25]:
# 시스템 프롬프트 수정
system_prompt = """
당신은 구조화 되지 않은 데이터를 JSON 형식으로 변환하는 AI입니다.
사용자가 과일에 대한 설명을 제공하면, 과일 이름과 전체 과일 수, 출처를 JSON 형식으로 추출하세요.
"""

In [26]:
response = client.responses.create(
  model="gpt-5-nano",
  input=[
    {
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": system_prompt
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": user_prompt
        }
      ]
    }
  ],
)

In [27]:
# 응답 출력
print(response.output_text)

{
  "fruit_names": ["사과","배","귤","포도","수박","참외","딸기","감","복숭아"],
  "total_fruit_count": 9,
  "source": "00뉴스"
}


잘하긴 했는데 중첩되진 않았네..

### 4.3 완성: 완벽한 JSON 출력 강제하기
- OpenAI `구조화된 출력 (Structured Outputs)` 기능 활용
- 모델의 응답을 우리가 원하는 JSON으로 강제하여, 예측 가능하고 안정적인 데이터를 얻어내는 OpenAI의 제공 기능
> https://platform.openai.com/docs/guides/structured-outputs

In [28]:
# 공식문서 스키마 붙여넣기
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

[참고] Pydantic 모델과 타입 힌트
1. 파이썬 타입 힌트 (Python Type Hint)

  - `name: str` 와 같이 변수명 뒤에 콜론(`:`)과 타입을 붙이는 것을 **타입 힌트(Type Hint)**라고 합니다. 이는 파이썬 3.5 버전부터 도입된 표준 문법입니다.

  - **역할**: 이 변수에 어떤 타입의 데이터가 들어갈 **예정인지**를 명시하여 코드의 가독성을 높이고, 개발 도구가 코드의 오류를 미리 파악할 수 있도록 돕습니다.

  - `name: str`: `name` 변수에는 문자열(`str`) 타입이 와야 합니다.

  - `participants: list[str]`: `participants` 변수에는 문자열(`str`)을 요소로 갖는 리스트(`list`)가 와야 합니다.

2. Pydantic의 역할
Pydantic은 바로 이 타입 힌트를 활용하여, **실제 프로그램 실행 시간에 데이터의 유효성을 검사하고 타입을 강제**하는 외부 라이브러리입니다.

  - `BaseModel`을 상속받은 클래스는 다음과 같은 기능을 갖게 됩니다.

  - **타입 강제**: 만약 `name`에 문자열이 아닌 숫자(`123`)를 넣으려고 하면, Pydantic이 에러를 발생시켜 잘못된 데이터가 들어오는 것을 막아줍니다.

  - **데이터 유효성 검사**: "이 필드는 반드시 이메일 형식이어야 한다" 또는 "이 숫자는 `0`보다 커야 한다"와 같은 복잡한 규칙도 추가할 수 있습니다.

In [29]:
# API 코드 붙여넣기
response = client.responses.parse(
    model="gpt-5-nano",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

In [30]:
# 응답 출력
print(response.output_text)

{"name":"Science fair","date":"Friday","participants":["Alice","Bob"]}


In [31]:
# 나만의 과일 정보 JSON 스키마 작성
from typing import List

# 1. 개별 과일의 구조를 정의하는 모델
# JSON 배열 안에 있는 각 객체에 해당합니다.
class Fruit(BaseModel):
    """
    'fruit_name'이라는 키를 가지며, 그 값은 반드시 문자열(str)
    'season'이라는 키를 가지며, 그 값은 반드시 문자열(str)
    """
    fruit_name: str
    season: str

# 2. 최종 JSON 응답의 전체 구조를 정의하는 모델
# 최상위 키 'fruits'가 Fruit 모델의 리스트를 값으로 가집니다.
class FruitList(BaseModel):
    """
    'fruits'라는 키를 가지며, 그 값은 반드시 Fruit 객체들의 리스트(List[Fruit])
    'num_of_fruits'라는 키를 가지며, 그 값은 반드시 정수(int)
    'ref'라는 키를 가지며, 그 값은 반드시 문자열(str)
    """
    fruits: List[Fruit]
    num_of_fruits: int
    ref: str

In [32]:
# 시스템 프롬프트 수정
system_prompt = """
당신은 구조화 되지 않은 데이터를 구조화된 형식으로 변환하는 AI입니다.
사용자가 과일에 대한 설명을 제공하면 각 과일에 대한 상세 정보를 추출하세요.
"""

In [33]:
# 사전에 정의한 스키마(과일, 계절, 과일 개수, 출처)을 이용하여 API 호출 만들기
response = client.responses.parse(
    model="gpt-5-nano",
    input=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    # JSON 스키마 지정
    text_format=FruitList
)

In [34]:
# 응답 출력
from pprint import pprint

parsed_data = json.loads(response.output_text)

pprint(parsed_data)

{'fruits': [{'fruit_name': '사과', 'season': '연중'},
            {'fruit_name': '배', 'season': '가을'},
            {'fruit_name': '귤', 'season': '겨울'},
            {'fruit_name': '포도', 'season': '여름-가을'},
            {'fruit_name': '수박', 'season': '여름'},
            {'fruit_name': '참외', 'season': '여름'},
            {'fruit_name': '딸기', 'season': '봄'},
            {'fruit_name': '감', 'season': '가을'},
            {'fruit_name': '복숭아', 'season': '여름'}],
 'num_of_fruits': 9,
 'ref': '00뉴스'}
